In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))

In [ ]:
source(paste0(HELP_DIR, "shortcuts.r"))

In [ ]:
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(patchwork)

In [ ]:
extra_theme <- 
theme(legend.position = "none") + 
  theme(axis.title.x = element_blank(),axis.title.y = element_blank()) + 
  theme(axis.text.y = element_text(size = 12), plot.title = element_text(size = 13)) 

# Prepare data for analysis

- Prepared categorical markers

In [ ]:
fisher_base <- fread(paste0(SHARE_DIR, "fisher_base.csv"))

In [ ]:
univariate_results <- 
fread(paste0(SHARE_DIR, "1_run_fishers_exact.csv")) %>% 
 mu(or = ifelse(or == "Inf", exp(5), or), 
    p_bh = p.adjust(fisher_pval, method = "BH"),  
    p_by = p.adjust(fisher_pval, method = "BY"))

- Barplot of Significance

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 10)

In [ ]:
threshold <- .01

- Organize the cohort names

In [ ]:
cohorts_map <- fread(paste0(SHARE_DIR, "cohort_names_map.csv"))

In [ ]:
base <- 
univariate_results %>% 
 ij(cohorts_map %>% se(cohortGo, cohortGoo, cohortGooo, pan), by = "cohortGo") %>% 
 mu(cohortGoo = factor(cohortGoo, levels = cohorts_map$cohortGoo[order(cohorts_map$ct)]))

In [ ]:
base$cohortGooo <-
factor(base$cohortGooo, 
       levels = base %>% gb(cohortGooo) %>% su(max_sig = max(-log10(fisher_pval))) %>% ar(desc(max_sig)) %>% pu(cohortGooo))

In [ ]:
#base %>% fi(direction == "Non-Response")

# 2 - Barplots for different cohorts

In [ ]:
plter <- function(d_base, title = "X", text_size = 4){
 dd <-   
 d_base %>% 
  gb(cohortGoo) %>% 
  su( p2 = mean(fisher_pval < threshold), p1 = mean(p_bh < threshold)) %>% 
  mu( p2 = p2 - p1) %>% 
  ga(gp, val, -cohortGoo)   

 dd_tot <- dd %>% gb(cohortGoo) %>% su(tot = sum(val))
    
 dd %>%
  ggplot(aes(y = cohortGoo, x = val, fill = gp)) + geom_bar(stat = "identity", color = "black") +   
  geom_vline(xintercept = threshold, color = "black", alpha = .3)+ 
  go_theme + extra_theme + labs(title = title) + scale_x_continuous(labels = scales::percent, expand = expansion(c(0, .3))) +
  geom_text(data = dd_tot, aes(y = cohortGoo, x = tot, label = paste0(round(signif(100*tot,2),1), "%")), inherit.aes = FALSE, hjust = -.2, size = 4) 
}

In [ ]:
p1 <- plter(base %>% fi(!pan, group == "treatment"), "Cohort Specific: Drug Treatment")
p2 <- plter(base %>% fi(!pan, group == "mechanism"), "Cohort Specific: Drug Mechanism") 
p3 <- plter(base %>% fi(!pan, group == "type"), "Cohort Specific: Drug Type") #+ labs(title = "Cohort Specific: Drug Type", x = "Number of Patients") + go_theme + extra_theme + theme(legend.position = "none",axis.title.y = element_blank()) 
p4 <- plter(base %>% fi(pan, group == "treatment"), "Pan-Cancer: Drug Treatment") 
p5 <- plter(base %>% fi(pan, group == "mechanism"), "Pan-Cancer: Drug Mechanism") 
p6 <- plter(base %>% fi(pan, group == "type"), "Pan-Cancer: Drug Type")  #+ labs(title = "Pan-Cancer: Drug Type", x = "Number of Patients") + go_theme + extra_theme + theme(legend.position = "none",axis.title.y = element_blank()) + scale_x_continuous(expand = expansion(c(0, 0.2))) 

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 10)

In [ ]:
cohort <- (p1 / p2 / p3 ) + plot_layout(heights = c(5, 7, 8)) 

In [ ]:
pan_cancer <- (p4 / p5 / p6 ) + plot_layout(heights = c(6, 6, 3)) 

In [ ]:
oo = (cohort | pan_cancer ) + plot_layout(widths = c(1,1)) #+ plot_annotation(title = "Overall Title", subtitle = "Subtitle here")

In [ ]:
oo 

In [ ]:
ggsave( "sig_cohorts.png", plot = oo, width = 12, height = 10)

# 3 - Volcano Plots

- Volcano Go

In [ ]:
threshold <- base %>% gb(mt = p_bh < .01) %>% su(threshold = max(fisher_pval)) %>% fi(mt) %>% ug() %>% pu(threshold)

In [ ]:
extra_theme <- theme(legend.position = "none") + theme(axis.title.x = element_blank(), axis.title.y = element_blank()) #+ 
#  theme(axis.text.y = element_text(size = 12), plot.title = element_text(size = 13)) 

In [ ]:
#base %>% ar(or)

In [ ]:
volcano <- function(df, title = "here", cols = 7){
 df %>% 
   ggplot( aes(x = log(or + .01), y = -log10(fisher_pval), alpha = (fisher_pval < .01 ), color = (or ==0))) + 
   geom_point() + 
   facet_wrap(~cohortGooo, ncol = cols) + 
   scale_alpha_manual(values = c(.01, 1)) + 
   scale_color_manual(values = c("black", "red")) + 
    
   go_theme + extra_theme + 
   geom_hline(yintercept = -log10(.01), alpha = .3, color = "orange") +
   geom_hline(yintercept = -log10(threshold), alpha = .7, color = "red") +
   labs(x = "Log (Odds Ratio)", y = "-Log10(p-value)", title = title) 
}

In [ ]:
p1 <- volcano(base %>% fi(!pan, group == "treatment"), "Cohort Specific: Drug Treatment", cols = 5)
p2 <- volcano(base %>% fi(!pan, group == "mechanism"), "Cohort Specific: Drug Mechanism") 
p3 <- volcano(base %>% fi(!pan, group == "type"), "Cohort Specific: Drug Type") 
p4 <- volcano(base %>% fi(pan, group == "treatment"), "Pan-Cancer: Drug Treatment") 
p5 <- volcano(base %>% fi(pan, group == "mechanism"), "Pan-Cancer: Drug Mechanism", cols = 6) 
p6 <- volcano(base %>% fi(pan, group == "type"), "Pan-Cancer: Drug Type") 

- Together

In [ ]:
cohort <- (p1 / p2 / p3 ) + plot_layout(heights = c(5, 5, 5)) 

In [ ]:
pan_cancer <- (p4 / p5 / p6 ) + plot_layout(heights = c(5, 5, 3)) 

In [ ]:
oo = (cohort | pan_cancer ) + plot_layout(widths = c(1,1)) 

In [ ]:
options(repr.plot.height = 11, repr.plot.width =20) 
oo

In [ ]:
ggsave( "volcanoes.png", plot = oo, width = 20, height = 11)

- Correlation Plot